In [6]:
import torch
import numpy as np
from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType
import pandas as pd
from pathlib import Path
import evaluate

# =============== КОНФИГУРАЦИЯ ===============
class Config:
    input_dir = "../rag_data/Pages/inputs"
    output_dir = "../rag_data/Pages/targets"
    model_save_dir = "fast_fine_tuned_t5"
    model_name = "cointegrated/rut5-small"
    batch_size = 2
    gradient_accumulation_steps = 4
    num_epochs = 15
    learning_rate = 5e-5
    max_input_length = 256
    max_target_length = 512
    prefix = "Cгенерируй описание продукта на основе текста карточки товара, разделив его особенности переносами строки, игнорируй артефакты вроде меню сайта: "
    
    # LoRA параметры
    lora_r = 16
    lora_alpha = 32
    lora_dropout = 0.1
    
    Path(model_save_dir).mkdir(parents=True, exist_ok=True)

# =============== ЗАГРУЗКА ДАННЫХ ===============
def load_data():
    pairs = []
    input_files = list(Path(Config.input_dir).glob("*.txt"))
    output_files = list(Path(Config.output_dir).glob("*.txt"))
    
    input_dict = {f.stem: f for f in input_files}
    output_dict = {f.stem: f for f in output_files}
    common_stems = set(input_dict.keys()) & set(output_dict.keys())
    
    for stem in list(common_stems)[:25]:
        try:
            with open(input_dict[stem], 'r', encoding='utf-8') as f:
                input_text = f.read().strip()
            with open(output_dict[stem], 'r', encoding='utf-8') as f:
                output_text = f.read().strip()
            
            if input_text and output_text:
                pairs.append({'input': input_text, 'target': output_text})
        except:
            continue
    
    print(f"Загружено {len(pairs)} примеров")
    
    if pairs:
        print("\nПример данных:")
        print(f"Вход: {pairs[0]['input'][:100]}...")
        print(f"Цель: {pairs[0]['target'][:100]}...")
    
    return pairs

# =============== ПОДГОТОВКА МОДЕЛИ ===============
def prepare_model():
    print(f"\nЗагрузка модели {Config.model_name}...")
    tokenizer = T5Tokenizer.from_pretrained(Config.model_name)
    model = T5ForConditionalGeneration.from_pretrained(Config.model_name)
    
    # Настройка LoRA
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        r=Config.lora_r,
        lora_alpha=Config.lora_alpha,
        lora_dropout=Config.lora_dropout,
        target_modules=["q", "v"],
        bias="none",
    )
    
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    
    return model, tokenizer

# =============== ПОДГОТОВКА ДАТАСЕТА ===============
def prepare_datasets(tokenizer, pairs):
    def preprocess_function(examples):
        inputs = [Config.prefix + text for text in examples['input']]
        targets = examples['target']
        
        # Токенизация входов
        model_inputs = tokenizer(
            inputs,
            max_length=Config.max_input_length,
            truncation=True,
            padding=False
        )
        
        # Токенизация целей
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                targets,
                max_length=Config.max_target_length,
                truncation=True,
                padding=False
            )
        
        # Заменяем pad_token_id на -100 для игнорирования в loss
        labels_input_ids = labels["input_ids"]
        for i in range(len(labels_input_ids)):
            labels_input_ids[i] = [
                (token_id if token_id != tokenizer.pad_token_id else -100) 
                for token_id in labels_input_ids[i]
            ]
        
        model_inputs["labels"] = labels_input_ids
        return model_inputs
    
    df = pd.DataFrame(pairs)
    dataset = Dataset.from_pandas(df)
    
    split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
    
    tokenized_datasets = split_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=split_dataset["train"].column_names
    )
    
    print(f"\nРазмеры датасетов:")
    print(f"Тренировочных: {len(tokenized_datasets['train'])}")
    print(f"Валидационных: {len(tokenized_datasets['test'])}")
    
    return tokenized_datasets

# =============== ОСНОВНОЕ ОБУЧЕНИЕ ===============
def train():
    # Загружаем данные
    pairs = load_data()
    if len(pairs) == 0:
        print("Нет данных для обучения!")
        return None, None
    
    # Подготавливаем модель и токенизатор
    model, tokenizer = prepare_model()
    
    # Подготавливаем датасет
    tokenized_datasets = prepare_datasets(tokenizer, pairs)
    
    # Определяем функцию compute_metrics ВНУТРИ функции train,
    # чтобы она имела доступ к переменной tokenizer
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        
        # Декодируем предсказания
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        
        # Заменяем -100 на pad_token_id для декодирования
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        
        # Вычисляем ROUGE метрики
        rouge = evaluate.load("rouge")
        
        result = rouge.compute(
            predictions=decoded_preds,
            references=decoded_labels,
            use_stemmer=True,
            use_aggregator=True
        )
        
        return {
            "rouge1": round(result["rouge1"], 4),
            "rouge2": round(result["rouge2"], 4),
            "rougeL": round(result["rougeL"], 4),
        }
    
    # Аргументы обучения
    training_args = Seq2SeqTrainingArguments(
        output_dir=Config.model_save_dir,
        overwrite_output_dir=True,
        per_device_train_batch_size=Config.batch_size,
        per_device_eval_batch_size=Config.batch_size,
        gradient_accumulation_steps=Config.gradient_accumulation_steps,
        num_train_epochs=Config.num_epochs,
        learning_rate=Config.learning_rate,
        lr_scheduler_type="linear",
        warmup_ratio=0.1,
        weight_decay=0.01,
        
        eval_strategy="steps",
        eval_steps=10,
        save_strategy="steps",
        save_steps=10,
        save_total_limit=2,
        load_best_model_at_end=False,
        
        predict_with_generate=True,
        generation_max_length=Config.max_target_length,
        generation_num_beams=2,
        
        logging_dir=f"{Config.model_save_dir}/logs",
        logging_steps=5,
        report_to="none",
        remove_unused_columns=False,
        label_names=["labels"],
        
        fp16=False,
        gradient_checkpointing=False,
    )
    
    # Коллатор данных
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        padding=True,
    )
    
    # Тренер
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"] if len(tokenized_datasets["test"]) > 0 else None,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    
    # Запуск обучения
    print("\n" + "="*60)
    print("НАЧИНАЕМ ОБУЧЕНИЕ")
    print("="*60)
    
    try:
        train_result = trainer.train()
        
        # Сохраняем модель
        model.save_pretrained(Config.model_save_dir)
        tokenizer.save_pretrained(Config.model_save_dir)
        print(f"\n✓ Модель сохранена в {Config.model_save_dir}")
        
        return model, tokenizer
        
    except Exception as e:
        print(f"\n✗ Ошибка при обучении: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# =============== ТЕСТИРОВАНИЕ ===============
def test_generation(model, tokenizer, text=None):
    """Тестирование генерации"""
    if text is None:
        text = "Стиральная машина"
    
    model.eval()
    input_text = Config.prefix + text
    
    inputs = tokenizer(
        input_text,
        max_length=Config.max_input_length,
        truncation=True,
        return_tensors="pt"
    )
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=Config.max_target_length,
            num_beams=3,
            temperature=0.9,
            do_sample=True,
            top_p=0.95,
            early_stopping=True,
            repetition_penalty=1.1,
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated

# =============== ЗАПУСК В JUPYTER ===============
if __name__ == "__main__":
    # Запускаем обучение
    model, tokenizer = train()
    
    # Тестируем если обучение успешно
    if model and tokenizer:
        print("\n" + "="*60)
        print("ТЕСТИРОВАНИЕ")
        print("="*60)
        
        test_texts = [
            """
                
Официальный дистрибьютор
0

Я ищу...
Главная 
Каталог 
Вытяжки 
Встраиваемые вытяжки 
Встраиваемая вытяжка De Dietrich DHT3622X
Духовые шкафы
Духовые шкафы
Компактные духовые шкафы
Духовые шкафы с паром
Компактные духовые шкафы с паром
Паровые шкафы
Варочные панели
Индукционные варочные панели
Варочные панели Домино
Электрические варочные панели
Газовые варочные панели
Комбинированные варочные панели
Вытяжки
Настенные вытяжки
Встраиваемые вытяжки
Островные вытяжки
Встраиваемые в столешницу вытяжки
Холодильные и морозильные шкафы
Встраиваемый холодильный шкаф
Винные шкафы
Встраиваемый морозильный шкаф
Встраиваемый холодильно-морозильный шкаф
Отдельностоящие холодильные шкафы
Посудомоечные машины
Микроволновые печи
Кофемашины
Вакууматоры и подогреватели
Стирально-сушильная техника
Аксессуары
Outlet и НКТ
Встраиваемая вытяжка De Dietrich DHT3622X
ALT
ALT
ALT
Варианты дизайна
Встраиваемая вытяжка De Dietrich DHT3622X
Встраиваемая вытяжка De Dietrich DHT3622X
Сравнение
62 990 ₽
Товар под заказ
Доставка: БЕСПЛАТНО ДО КВАРТИРЫ
заказ



Описание
Характеристики
Документы
Встраиваемая вытяжка De Dietrich DHT3622X

Механическое управление с курсором
Маркировка энергопотребления: класс B
Функция "Вкл./Выкл."
Функция "Boost"
Максимальная производительность: 425 м3/ч
Уровень шума: мин: 57 дБ(A), макс.: 69 дБ(A)
3 уровня мощности, включая Boost
Независимое освещение
2 x 2,5 Вт светодиодное освещение


Посмотреть все вытяжки коллекции PLATINUM

Посмотреть все товары коллекции PLATINUM



Режим BOOST
Режим BOOST
Выбирая этот режим вы мгновенно устанавливаете максимальную производительность, чтобы самым эффективным образом удалять большие количества испарений.
Светодиодное освещение
Светодиодное освещение
Этот метод освещения выделяется тем, что потребляет почти на 85% меньше электроэнергии и служит в 20 раз дольше, чем лампа накаливания. Система эффективно освещает все кастрюли на вашей плите и обеспечивает вам мягкое освещение, идеально подходящее для ужина.



zakaz@dedietrich-shop.ru

Мы в соцсетях:
TELEGRAM	ВКонтакте
©2026. Магазин французской бытовой техники De Dietrich. Все права защищены

            """
        ]
        
        for text in test_texts:
            print(f"\nВход: {text}")
            result = test_generation(model, tokenizer, text)
            print(f"Результат:\n{result}")

Загружено 24 примеров

Пример данных:
Вход: Официальный дистрибьютор
0

Я ищу...
Главная 
Каталог 
Вытяжки 
Встраиваемые вытяжки 
Встраиваемая в...
Цель: Встраиваемая вытяжка De Dietrich DHT3622X
Механическое управление с курсором
класс энергопотребления...

Загрузка модели cointegrated/rut5-small...


You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


trainable params: 688,128 || all params: 65,332,608 || trainable%: 1.0533


Map: 100%|██████████| 5/5 [00:00<00:00, 92.21 examples/s]
C:\Users\loban\AppData\Local\Temp\ipykernel_12980\2105122154.py:224: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(



Размеры датасетов:
Тренировочных: 19
Валидационных: 5

НАЧИНАЕМ ОБУЧЕНИЕ


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
10,5.990600,5.466343,0.012900,0.000000,0.012900
20,5.877100,5.420467,0.012900,0.000000,0.012900
30,5.734800,5.384649,0.012900,0.000000,0.012900
40,5.790400,5.365154,0.012900,0.000000,0.012900


You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
c:\Users\loban\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
c:\Users\loban\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
c:\Users\loban\AppData\Local\Programs\Python\Python3


✓ Модель сохранена в fast_fine_tuned_t5

ТЕСТИРОВАНИЕ

Вход: 
                
Официальный дистрибьютор
0

Я ищу...
Главная 
Каталог 
Вытяжки 
Встраиваемые вытяжки 
Встраиваемая вытяжка De Dietrich DHT3622X
Духовые шкафы
Духовые шкафы
Компактные духовые шкафы
Духовые шкафы с паром
Компактные духовые шкафы с паром
Паровые шкафы
Варочные панели
Индукционные варочные панели
Варочные панели Домино
Электрические варочные панели
Газовые варочные панели
Комбинированные варочные панели
Вытяжки
Настенные вытяжки
Встраиваемые вытяжки
Островные вытяжки
Встраиваемые в столешницу вытяжки
Холодильные и морозильные шкафы
Встраиваемый холодильный шкаф
Винные шкафы
Встраиваемый морозильный шкаф
Встраиваемый холодильно-морозильный шкаф
Отдельностоящие холодильные шкафы
Посудомоечные машины
Микроволновые печи
Кофемашины
Вакууматоры и подогреватели
Стирально-сушильная техника
Аксессуары
Outlet и НКТ
Встраиваемая вытяжка De Dietrich DHT3622X
ALT
ALT
ALT
Варианты дизайна
Встраиваемая вытяжка De Dietrich DH

In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from pathlib import Path

# =============== КОНФИГУРАЦИЯ ===============
MODEL_PATH = "fast_fine_tuned_t5"  # Путь к обученной модели
INPUT_FILE = "../rag_data/Pages/inputs/page_1_DHT3622X.txt"  # Входной файл
OUTPUT_FILE = "output.txt"  # Выходной файл

# Параметры генерации
GENERATION_PARAMS = {
    "max_length": 256,
    "num_beams": 4,
    "temperature": 0.7,
    "do_sample": True,
    "top_p": 0.9,
    "repetition_penalty": 1.2,
}

# =============== ЗАГРУЗКА МОДЕЛИ С LoRA ===============
def load_lora_model(model_path):
    """Загрузка модели с адаптерами LoRA"""
    print(f"Загрузка модели из {model_path}...")
    
    # Загружаем конфигурацию LoRA
    peft_config = PeftConfig.from_pretrained(model_path)
    
    # Загружаем базовую модель
    base_model = AutoModelForSeq2SeqLM.from_pretrained(
        peft_config.base_model_name_or_path
    )
    
    # Загружаем модель с адаптерами LoRA
    model = PeftModel.from_pretrained(base_model, model_path)
    
    # Загружаем токенизатор
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    # Переводим модель в режим инференса
    model.eval()
    
    # Перемещаем на GPU если доступно
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    print(f"Модель загружена на устройство: {device}")
    print(f"Размер модели: {model.num_parameters():,} параметров")
    
    return model, tokenizer, device

# =============== ЧТЕНИЕ ВХОДНОГО ТЕКСТА ===============
def read_input_text(file_path):
    """Чтение текста из файла"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        print(f"Прочитан файл: {file_path}")
        print(f"Длина текста: {len(text)} символов")
        return text
    except Exception as e:
        print(f"Ошибка чтения файла: {e}")
        return None

# =============== ГЕНЕРАЦИЯ ТЕКСТА ===============
def generate_text(model, tokenizer, device, input_text, prefix="summarize: "):
    """Генерация текста моделью"""
    # Добавляем префикс (такой же, как при обучении)
    processed_text = prefix + input_text
    
    # Токенизируем входной текст
    inputs = tokenizer(
        processed_text,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    
    # Перемещаем на то же устройство, что и модель
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Генерируем текст
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **GENERATION_PARAMS
        )
    
    # Декодируем результат
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

# =============== СОХРАНЕНИЕ РЕЗУЛЬТАТА ===============
def save_output(text, file_path):
    """Сохранение сгенерированного текста в файл"""
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"\nРезультат сохранен в: {file_path}")
        print(f"Длина сгенерированного текста: {len(text)} символов")
        return True
    except Exception as e:
        print(f"Ошибка сохранения файла: {e}")
        return False

# =============== ОСНОВНАЯ ФУНКЦИЯ ===============
def main():
    print("=" * 60)
    print("ГЕНЕРАЦИЯ ТЕКСТА С ОБУЧЕННОЙ МОДЕЛЬЮ T5-LoRA")
    print("=" * 60)
    
    # 1. Проверяем наличие файлов
    if not Path(MODEL_PATH).exists():
        print(f"Ошибка: модель не найдена по пути {MODEL_PATH}")
        return
    
    if not Path(INPUT_FILE).exists():
        print(f"Ошибка: входной файл не найден {INPUT_FILE}")
        return
    
    # 2. Загружаем модель
    model, tokenizer, device = load_lora_model(MODEL_PATH)
    
    # 3. Читаем входной текст
    input_text = read_input_text(INPUT_FILE)
    if not input_text:
        return
    
    # 4. Генерируем текст
    print("\nГенерация текста...")
    generated_text = generate_text(model, tokenizer, device, input_text)
    
    # 5. Выводим результаты
    print("\n" + "=" * 60)
    print("РЕЗУЛЬТАТЫ:")
    print("=" * 60)
    print(f"\nВходной текст (первые 300 символов):")
    print("-" * 40)
    print(input_text[:300] + "..." if len(input_text) > 300 else input_text)
    
    print(f"\n\nСгенерированный текст:")
    print("-" * 40)
    print(generated_text)
    
    # 6. Сохраняем результат
    save_output(generated_text, OUTPUT_FILE)
    
    print("\n" + "=" * 60)
    print("ГЕНЕРАЦИЯ ЗАВЕРШЕНА УСПЕШНО!")
    print("=" * 60)

# =============== ЗАПУСК ===============
if __name__ == "__main__":
    main()

c:\Users\loban\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ГЕНЕРАЦИЯ ТЕКСТА С ОБУЧЕННОЙ МОДЕЛЬЮ T5-LoRA
Загрузка модели из fast_fine_tuned_t5...


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
c:\Users\loban\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Модель загружена на устройство: cpu
Размер модели: 65,332,608 параметров
Прочитан файл: ../rag_data/Pages/inputs/page_1_DHT3622X.txt
Длина текста: 2114 символов

Генерация текста...

РЕЗУЛЬТАТЫ:

Входной текст (первые 300 символов):
----------------------------------------
Официальный дистрибьютор
0

Я ищу...
Главная 
Каталог 
Вытяжки 
Встраиваемые вытяжки 
Встраиваемая вытяжка De Dietrich DHT3622X
Духовые шкафы
Духовые шкафы
Компактные духовые шкафы
Духовые шкафы с паром
Компактные духовые шкафы с паром
Паровые шкафы
Варочные панели
Индукционные варочные панели
Вароч...


Сгенерированный текст:
----------------------------------------
Стирально-сушильная техника Аксессуары Оutlet и НКТ встраиваемые вытяжки Встраиваемые в столешницу вытяжки Холодильные и морозильные шкафы Посудомоечные машины Микроволновые печи Кофемашины Вакууматоры и подогреватели Стирально-сушильная техника Аксессуары Оutlet и НКТ Встраиваемые вытяжки Встраиваемые вытяжки Встраиваемые встраиваемые вытяжки Холодильны

In [6]:
from openai import OpenAI

def extract_product_info(file_path):
    # Читаем содержимое файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text_content = file.read()
    
    # Подготавливаем системное сообщение и пользовательский запрос
    system_message = "Ты помощник для извлечения информации о товаре из текста веб-страницы."
    user_query = """Перед тобой текст, скопированный с веб-страницы карточки товара, убери из него все артефакты сайта типа хэдера, падера и пунктов меню, чтобы осталось только наименование товара в первой строке и разделённые переносами строк свойства товара

Вот текст:
{text}""".format(text=text_content)
    
    # Подключаемся к API
    client = OpenAI(
        api_key="nK1FfIy_yI90TPVqIafoc7Pd38i-gBD6",
        base_url="https://chat.immers.cloud/v1/endpoints/gpt-oss-20b/generate/",
    )
    
    # Отправляем запрос
    chat_response = client.chat.completions.create(
        model="gpt-oss-20b",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_query},
        ],
        temperature=0.1  # Низкая температура для более детерминированного ответа
    )
    
    # Возвращаем результат
    return chat_response.choices[0].message.content

# Использование
result = extract_product_info("../rag_data/Pages/inputs/page_1_DHT3622X.txt")  # Укажите путь к вашему файлу
with open("output.txt", 'w', encoding='utf-8') as f:
    f.write(result)